In [10]:
import pandas as pd
#pip install openpyxl

dfs = pd.read_excel('https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx.xlsx', sheet_name=None)

print(dfs.keys())





dict_keys(['ad1', 'episodes', 'header', 'missing', 'oc2', 'oc3', 'placed_for_adoption', 'previous_permenance', 'reviews', 'uasc'])


In [ ]:
# .notna()
# 102 - date of birth is not a valid date
dfs['header']['DOB'] = pd.to_datetime(dfs['header']['DOB'], format="%Y-%m-%d", errors='coerce')

invalid_dob = dfs['header']['DOB'].notna()

invalid_rows = dfs['header'][invalid_dob]

invalid_rows

# 105 - Date of Local Authority's (LA) decision that a child should be placed for adoption is not a valid date."

In [9]:
# .isin()
# 103 - The ethnicity code is either not valid or has not been entered.
header = dfs['header']

codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
    ]

valid_rows = header['ETHNIC'].isin(codes)

invalid_rows = header[~valid_rows]

invalid_rows

# 104 - Data entry to record the status of former carer(s) of an adopted child is invalid.
code_list = ["0", "1"]


,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB


In [15]:
# 184 - Date of decision that a child should be placed for adoption is before the child was born.

header = dfs['header']
pfa = dfs['placed_for_adoption']

df = pd.merge(pfa.reset_index(), header, how='left', on='CHILD', suffixes=[None, "_P4A"])

df["DATE_PLACED"] = pd.to_datetime(
            df["DATE_PLACED"], format="%d/%m/%Y", errors="coerce"
        )
df["DOB"] = pd.to_datetime(
    df["DOB"], format="%d/%m/%Y", errors="coerce"
)

mask = (df["DATE_PLACED"] >= df["DOB"]) | df[
            "DATE_PLACED"
        ].isna()

validation_error = df[~mask]

validation_error


# 302 - First episode starts before child was born.

,index,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,SEX,DOB_P4A,ETHNIC,UPN,MOTHER,MC_DOB


In [23]:
# 188 - Child is aged under 4 years at the end of the year but a Strengths and Difficulties (SDQ) score or a reason for no SDQ score has been completed
oc2 = dfs['oc2']
collection_end = pd.to_datetime('30/04/2020', dayfirst=True)

oc2['DOB'] = pd.to_datetime(oc2['DOB'], format="%d/%m/%Y", errors="coerce")

oc2["4th_bday"] = oc2["DOB"] + pd.DateOffset(years=4)
error_mask = (oc2["4th_bday"] > collection_end) & (
    oc2["SDQ_SCORE"].notna() | oc2["SDQ_REASON"].notna())

errors = oc2[error_mask]
errors

# 189 - Child is aged 17 years or over at the beginning of the year, but an Strengths and Difficulties (SDQ) score or a reason for no Strengths and Difficulties (SDQ) score has been completed.

,CHILD,DOB,SDQ_SCORE,SDQ_REASON,CONVICTED,HEALTH_CHECK,IMMUNISATIONS,TEETH_CHECK,HEALTH_ASSESSMENT,SUBSTANCE_MISUSE,INTERVENTION_RECEIVED,INTERVENTION_OFFERED,4th_bday


In [ ]:
# 202 - The gender code conflicts with the gender already recorded for this child.
# 203 - Date of birth disagrees with the date of birth already recorded for this child.

In [ ]:
# 218 - "Ofsted Unique reference number (URN) is required."
# CHILD id is needed 